# Working with Distinct Values, $elemMatch, and Regex: Exercises

In [ ]:
from bson.regex import Regex
from pymongo import MongoClient

client = MongoClient()
db = client.nobel

## Never from there, but sometimes there at last

There are some recorded countries of death (`"diedCountry"`) that do not appear as a country of birth (`"bornCountry"`) for laureates. One such country is "East Germany".

- Return a set of all such countries as `countries`.

In [ ]:
# Countries recorded as countries of death but not as countries of birth
countries = set(db.laureates.distinct("diedCountry")) - set(db.laureates.distinct("bornCountry"))
print(countries)

## Countries of affiliation

We saw in the last exercise that countries can be associated with a laureate as their country of birth and as their country of death. For each prize a laureate received, they may also have been affiliated with an institution at the time, located in a country.

- Determine the number of distinct countries recorded as part of an affiliation for laureates' prizes. Save this as `count`.

In [ ]:
# The number of distinct countries of laureate affiliation for prizes
count = len(db.laureates.distinct("prizes.affiliations.country"))
print(count)

## Triple plays (mostly) all around

All prize categories but one – literature – have had prizes shared by three or more laureates.

- Save a filter document `criteria` that, when passed to `db.prizes.distinct`, returns all prize categories shared by three or more laureates. That is, `"laureates.2"` must exist for such documents.
- Save these prize categories as a Python `set` called `triple_play_categories`.
- Confirm via an assertion that "literature" is the only prize category with no prizes shared by three or more laureates.

In [ ]:
# Save a filter for prize documents with three or more laureates
criteria = {"laureates.2": {"$exists": True}}

# Save the set of distinct prize categories in documents satisfying the criteria
triple_play_categories = set(db.prizes.distinct("category", criteria))

# Confirm literature as the only category not satisfying the criteria.
assert set(db.prizes.distinct("category")) - triple_play_categories == {"literature"}

## Other sharing after World War II

Similar to the example in the lesson, what is the approximate ratio of the number of laureates who won an *unshared* prize in categories other than physics, chemistry, and medicine after World War II to the number of laureates who won a *shared* prize in categories other than physics, chemistry, and medicine after World War II?

- Save an `$elemMatch` filter `unshared` to count laureates with unshared prizes in categories other than ("not in") `["physics", "chemistry", "medicine"]` in or after 1945.
- Save an `$elemMatch` filter `shared` to count laureates with shared (i.e., "share" is not "1") prizes in categories other than `["physics", "chemistry", "medicine"]` in or after 1945.



In [ ]:
# Save a filter for laureates with unshared prizes
unshared = {
    "prizes": {"$elemMatch": {
        "category": {"$nin": ["physics", "chemistry", "medicine"]},
        "share": "1",
        "year": {"$gt": "1945"},
    }}}

# Save a filter for laureates with shared prizes
shared = {
    "prizes": {"$elemMatch": {
        "category": {"$nin": ["physics", "chemistry", "medicine"]},
        "share": {"$gt": "1"},
        "year": {"$gt": "1945"},
    }}}

ratio = db.laureates.count_documents(unshared) / db.laureates.count_documents(shared)
print(ratio)

## Organizations and prizes over time

How many organizations won prizes before 1945 versus in or after 1945?

- Save a filter `before` to count organization laureates with prizes won before 1945. Recall that organization status is encoded with the "gender" field, and that dot notation is needed to access a laureate's "year" field within its "prizes" array.
- Save a filter `in_or_after` to count organization laureates with prizes won in or after 1945.

In [ ]:
# Save a filter for organization laureates with prizes won before 1945
before = {
    "gender": "org",
    "prizes.year": {"$lt": "1945"}
    }

# Save a filter for organization laureates with prizes won in or after 1945
in_or_after = {
    "gender": "org",
    "prizes.year": {"$gte": "1945"}
    }

n_before = db.laureates.count_documents(before)
n_in_or_after = db.laureates.count_documents(in_or_after)
ratio = n_in_or_after / (n_in_or_after + n_before)
print(ratio)

## Germany, then and now

Just as we saw with Poland, there are laureates who were born somewhere that was in Germany at the time but is now not, and others born somewhere that was not in Germany at the time but now is.

- Use a regular expression object to filter for laureates with "Germany" in their "bornCountry" value.

In [ ]:
# Filter for laureates with "Germany" in their "bornCountry" value
criteria = {"bornCountry": Regex("Germany")}
print(set(db.laureates.distinct("bornCountry", criteria)))

- Use a regular expression object to filter for laureates with a "bornCountry" value starting with "Germany".

In [ ]:
# Filter for laureates with a "bornCountry" value starting with "Germany"
criteria = {"bornCountry": Regex("^Germany")}
print(set(db.laureates.distinct("bornCountry", criteria)))

- Use a regular expression object to filter for laureates born in what was at the time Germany but is now another country.

In [ ]:
# Fill in a string value to be sandwiched between the strings "^" and "now"
criteria = {"bornCountry": Regex("^" + "Germany \(" + "now")}
print(set(db.laureates.distinct("bornCountry", criteria)))

- Use a regular expression object to filter for laureates born in what is now Germany but at the time was another country.

In [ ]:
# Filter for currently-Germany countries of birth.
# Fill in a string value to be sandwiched between the strings "now" and "$"
criteria = {"bornCountry": Regex("now" + " Germany\)" + "$")}
print(set(db.laureates.distinct("bornCountry", criteria)))

## The prized transistor

Three people shared a Nobel prize "for their researches on semiconductors and their discovery of the transistor effect". We can filter on "transistor" as a substring of a laureate's "prizes.motivation" field value to find these laureates.

- Save a filter `criteria` that finds laureates with `prizes.motivation` values containing "transistor" as a substring. The substring can appear anywhere within the value, so no anchoring characters are needed.
- Save to `first` and `last` the field names corresponding to a laureate's first name and last name (i.e. "surname") so that we can print out the names of these laureates.



In [ ]:
# Save a filter for laureates with prize motivation values containing "transistor" as a substring
criteria = {"prizes.motivation": Regex("transistor")}

# Save the field names corresponding to a laureate's first name and last name
first, last = "firstname", "surname"
print([(laureate[first], laureate[last]) for laureate in db.laureates.find(criteria)])